# [1193. Monthly Transactions I](https://leetcode.com/problems/monthly-transactions-i/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Transactions

<pre>+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| country       | varchar |
| state         | enum    |
| amount        | int     |
| trans_date    | date    |
+---------------+---------+</pre>
id is the primary key of this table.
The table has information about incoming transactions.
The state column is an enum of type ("approved", "declined").


Write an SQL query to find for each month and country, the number of transactions and their total amount, the number of approved transactions and their total amount.

Return the result table in any order.

The query result format is in the following example.



Example 1:

Input:
Transactions table:
<pre>+------+---------+----------+--------+------------+
| id   | country | state    | amount | trans_date |
+------+---------+----------+--------+------------+
| 121  | US      | approved | 1000   | 2018-12-18 |
| 122  | US      | declined | 2000   | 2018-12-19 |
| 123  | US      | approved | 2000   | 2019-01-01 |
| 124  | DE      | approved | 2000   | 2019-01-07 |
+------+---------+----------+--------+------------+</pre>
Output:
<pre>+----------+---------+-------------+----------------+--------------------+-----------------------+
| month    | country | trans_count | approved_count | trans_total_amount | approved_total_amount |
+----------+---------+-------------+----------------+--------------------+-----------------------+
| 2018-12  | US      | 2           | 1              | 3000               | 1000                  |
| 2019-01  | US      | 1           | 1              | 2000               | 2000                  |
| 2019-01  | DE      | 1           | 1              | 2000               | 2000                  |
+----------+---------+-------------+----------------+--------------------+-----------------------+</pre>

In [ ]:
# pandas schema

import pandas as pd

data = [[121, 'US', 'approved', 1000, '2018-12-18'], [122, 'US', 'declined', 2000, '2018-12-19'],
        [123, 'US', 'approved', 2000, '2019-01-01'], [124, 'DE', 'approved', 2000, '2019-01-07']]
transactions = pd.DataFrame(data, columns=['id', 'country', 'state', 'amount', 'trans_date']).astype(
    {'id': 'Int64', 'country': 'object', 'state': 'object', 'amount': 'Int64', 'trans_date': 'datetime64[ns]'})

# to spark dataframe

#to pyspark df

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

transactions_df = spark.createDataFrame(transactions)

In [ ]:
transactions

,id,country,state,amount,trans_date
0,121,US,approved,1000,2018-12-18
1,122,US,declined,2000,2018-12-19
2,123,US,approved,2000,2019-01-01
3,124,DE,approved,2000,2019-01-07


In [ ]:
# solving in spark dataframe

transactions_df.withColumn('month', substring('trans_date', 0, 7)) \
    .groupBy('month', 'country') \
    .agg(
      count('id').alias('trans_count'),
      sum(when(col('state') == 'approved', 1)).alias('approved_count'),
      sum('amount').alias('trans_total_amount'),
      sum(when(col('state') == 'approved', col('amount'))).alias('approved_total_amount')
    ).show()

+-------+-------+-----------+--------------+------------------+---------------------+
|  month|country|trans_count|approved_count|trans_total_amount|approved_total_amount|
+-------+-------+-----------+--------------+------------------+---------------------+
|2018-12|     US|          2|             1|              3000|                 1000|
|2019-01|     US|          1|             1|              2000|                 2000|
|2019-01|     DE|          1|             1|              2000|                 2000|
+-------+-------+-----------+--------------+------------------+---------------------+



In [ ]:
# solving in spark SQL

transactions_df.createOrReplaceTempView('transactions')

spark.sql('''
select
    left(trans_date,7) as month,
    country,
    count(id) as trans_count,
    sum(case when state='approved' then 1 end) as approved_count,
    sum(amount) as trans_total_amount,
    sum(case when state='approved' then amount end) as approved_total_amount
from transactions
group by month, country;

''').show()

+-------+-------+-----------+--------------+------------------+---------------------+
|  month|country|trans_count|approved_count|trans_total_amount|approved_total_amount|
+-------+-------+-----------+--------------+------------------+---------------------+
|2018-12|     US|          2|             1|              3000|                 1000|
|2019-01|     US|          1|             1|              2000|                 2000|
|2019-01|     DE|          1|             1|              2000|                 2000|
+-------+-------+-----------+--------------+------------------+---------------------+



In [ ]:
spark.stop()